In [10]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
#import maup
from zipfile import ZipFile
import numpy as np
import os
import shutil

# US President

## Final File - Load

In [11]:
pwd

'/Users/peterhorton/Documents/RDH/pdv/pdv-va/vest-va-2016'

In [12]:
final = gpd.read_file('./vest_va_2016/va_2016/va_2016_president.shp')
final

,COUNTYFP,LOCALITY,VTDST,PRECINCT,G16PREDCLI,G16PRERTRU,G16PRELJOH,G16PREIMCM,G16PREGSTE,G16PREOWRI,geometry
0,001,Accomack County,000101,Chincoteague,502,1285,31,12,8,6,"POLYGON Z ((-75.42507 37.89957 0.00000, -75.42..."
1,001,Accomack County,000201,Atlantic,198,598,15,8,1,1,"POLYGON Z ((-75.59978 37.87664 0.00000, -75.59..."
2,001,Accomack County,000202,Greenbackville,342,753,19,5,8,3,"POLYGON Z ((-75.49919 37.93416 0.00000, -75.49..."
3,001,Accomack County,000301,New Church,808,640,24,5,3,6,"POLYGON Z ((-75.64987 37.92702 0.00000, -75.64..."
4,001,Accomack County,000401,Bloxom,160,370,10,4,1,0,"POLYGON Z ((-75.71556 37.87513 0.00000, -75.71..."
...,...,...,...,...,...,...,...,...,...,...,...
2451,153,Prince William County,000302,Potomac,2382,512,54,32,16,25,"POLYGON Z ((-77.32258 38.58021 0.00000, -77.32..."
2452,179,Stafford County,000302,WIDEWATER,1562,1248,103,33,18,20,"POLYGON Z ((-77.39428 38.47458 0.00000, -77.39..."
2453,179,Stafford County,000401,AQUIA,555,303,24,14,6,6,"POLYGON Z ((-77.39221 38.48583 0.00000, -77.39..."
2454,195,Wise County,000302,East Stone Gap,297,1555,22,16,9,10,"POLYGON Z ((-82.80429 36.83206 0.00000, -82.80..."


# Election Results: US President

## Read in election results

In [13]:
df = pd.read_csv('./vest_va_2016/Virginia_Elections_Database__2016_President_General_Election_including_precincts.csv')

In [14]:
df.head()

,County/City,Ward,Pct,Hillary R. Clinton,Donald J. Trump,Gary Johnson,Evan McMullin,Jill Stein,All Others,Total Votes Cast
0,NaN,NaN,NaN,Democratic,Republican,Libertarian,Independent,Green,NaN,NaN
1,Accomack County,-,101 - Chincoteague,444,"1,188",26,10,7,5,"1,680"
2,Accomack County,-,201 - Atlantic,175,553,13,7,1,1,750
3,Accomack County,-,202 - Greenbackville,302,697,17,4,7,3,"1,030"
4,Accomack County,-,301 - New Church,713,592,21,4,3,5,"1,338"


In [15]:
df['LOCALITY'] = df['County/City']
df['G16PREDCLI'] = df['Hillary R. Clinton']
df['G16PRERTRU'] = df['Donald J. Trump']
df['G16PRELJOH'] = df['Gary Johnson'] 
df['G16PREIMCM'] = df['Evan McMullin']
df['G16PREGSTE'] = df['Jill Stein']
df['G16PREOWRI'] = df['All Others']
df = df.iloc[1:]
df = df.drop(['County/City', 'Hillary R. Clinton', 'Donald J. Trump', 'Gary Johnson', 'Evan McMullin', 'Jill Stein', 'Ward',  'All Others', 'Total Votes Cast'], axis = 1)

df = df.fillna(value = 0)
county_dict = pd.Series(final['COUNTYFP'].values, index = final['LOCALITY']).to_dict()
df['COUNTYFP'] = df['LOCALITY'].map(county_dict)

df['G16PREDCLI'] = df['G16PREDCLI'].map(lambda x: str(x).replace(',', ''))
df['G16PRERTRU'] = df['G16PRERTRU'].map(lambda x: str(x).replace(',', ''))
df['G16PRELJOH'] = df['G16PRELJOH'].map(lambda x: str(x).replace(',', ''))
df['G16PREIMCM'] = df['G16PREIMCM'].map(lambda x: str(x).replace(',', ''))
df['G16PREGSTE'] = df['G16PREGSTE'].map(lambda x: str(x).replace(',', ''))
df['G16PREOWRI'] = df['G16PREOWRI'].map(lambda x: str(x).replace(',', ''))

df['G16PREDCLI'] = df['G16PREDCLI'].astype(str).astype(float).astype(int)
df['G16PRERTRU'] = df['G16PRERTRU'].astype(str).astype(float).astype(int)
df['G16PRELJOH'] = df['G16PRELJOH'].astype(str).astype(float).astype(int)
df['G16PREIMCM'] = df['G16PREIMCM'].astype(str).astype(float).astype(int)
df['G16PREGSTE'] = df['G16PREGSTE'].astype(str).astype(float).astype(int)
df['G16PREOWRI'] = df['G16PREOWRI'].astype(str).astype(float).astype(int)

df.head()

,Pct,LOCALITY,G16PREDCLI,G16PRERTRU,G16PRELJOH,G16PREIMCM,G16PREGSTE,G16PREOWRI,COUNTYFP
1,101 - Chincoteague,Accomack County,444,1188,26,10,7,5,001
2,201 - Atlantic,Accomack County,175,553,13,7,1,1,001
3,202 - Greenbackville,Accomack County,302,697,17,4,7,3,001
4,301 - New Church,Accomack County,713,592,21,4,3,5,001
5,401 - Bloxom,Accomack County,141,342,9,3,1,0,001


In [16]:
df = df[df["LOCALITY"]!="TOTALS"]

In [17]:
column_list = ['G16PREDCLI', 'G16PRERTRU', 'G16PRELJOH', 'G16PREIMCM', 'G16PREGSTE', 'G16PREOWRI']

In [18]:
for val in column_list:
    vote_dif = df[val].sum()-final[val].sum()
    if (vote_dif == 0):
        print(val+": EQUAL")
    else:
        print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES")

G16PREDCLI: EQUAL
G16PRERTRU: EQUAL
G16PRELJOH: EQUAL
G16PREIMCM: EQUAL
G16PREGSTE: EQUAL
G16PREOWRI: EQUAL


In [19]:
print("Counties with differences printed below:")
diff_counties=[]
for i in column_list:
    diff = df.groupby(["LOCALITY"]).sum()[i]-final.groupby(["LOCALITY"]).sum()[i]
    for val in diff[diff != 0].index.values.tolist():
        if val not in diff_counties:
            diff_counties.append(val)
    if len(diff[diff != 0]!=0):
        print(diff[diff != 0].to_string(header=False))
print("")
print("All other races in all counties are equal")

Counties with differences printed below:

All other races in all counties are equal


## Re allocate absentee votes

### explore data

In [20]:
df[(df['Pct'].map(lambda x: 'Absentee' in str(x))) & (df['Pct'].map(lambda x: 'CD' in str(x)))] #| (df['Pct'].map(lambda x: 'Provisional' in str(x)))]

,Pct,LOCALITY,G16PREDCLI,G16PRERTRU,G16PRELJOH,G16PREIMCM,G16PREGSTE,G16PREOWRI,COUNTYFP
244,Ab - Central Absentee Precinct (CD 5),Bedford County,649,1246,43,22,7,13,019
245,Ab - Central Absentee Precinct (CD 6),Bedford County,556,1058,65,38,6,19,019
467,AB - Central Absentee Precinct (CD 3),Chesapeake City,4171,1847,120,51,32,35,550
468,Ab - Central Absentee Precinct (CD 4),Chesapeake City,4233,4316,257,79,33,52,550
546,Ab - Central Absentee Precinct (CD 4),Chesterfield County,2880,2952,139,50,11,38,041
547,Ab - Central Absentee Precinct (CD 7),Chesterfield County,6138,4997,465,150,51,104,041
910,Ab - Central Absentee Precinct (CD 10),Fairfax County,17488,8233,809,428,136,455,059
911,Ab - Central Absentee Precinct (CD 11),Fairfax County,40351,13165,1487,890,351,735,059
912,Ab - Central Absentee Precinct (CD 8),Fairfax County,26158,7795,869,562,239,509,059
949,Ab - Central Absentee Precinct (CD 1),Fauquier County,210,324,22,12,10,6,061


In [21]:
df[(df['Pct'].map(lambda x: 'Provisional' in str(x))) & (df['Pct'].map(lambda x: 'CD' in str(x)))] #| (df['Pct'].map(lambda x: 'Provisional' in str(x)))]

,Pct,LOCALITY,G16PREDCLI,G16PRERTRU,G16PRELJOH,G16PREIMCM,G16PREGSTE,G16PREOWRI,COUNTYFP
246,Provisional (CD 5),Bedford County,5,9,0,1,0,0,019
247,Provisional (CD 6),Bedford County,4,6,0,0,0,0,019
469,Provisional (CD 3),Chesapeake City,23,14,2,1,1,0,550
470,Provisional (CD 4),Chesapeake City,41,41,3,1,1,0,550
548,Provisional (CD 4),Chesterfield County,29,26,0,0,0,1,041
549,Provisional (CD 7),Chesterfield County,49,38,6,1,1,0,041
913,Provisional (CD 10),Fairfax County,72,38,8,0,0,3,059
914,Provisional (CD 11),Fairfax County,201,90,16,6,5,3,059
915,Provisional (CD 8),Fairfax County,187,55,11,11,0,2,059
951,Provisional (CD 1),Fauquier County,1,4,0,0,0,0,061


See that for county where Abs or Prov has cd on end, so do other precinct names
so, create list of the precincts that have cd necessary
then assign new county name based on that ID

In [22]:
cd_abs_prov_prec = df[((df['Pct'].map(lambda x: 'Absentee' in str(x))) &(df['Pct'].map(lambda x: 'CD' in str(x)))) | ((df['Pct'].map(lambda x: 'Provisional' in str(x))) & (df['Pct'].map(lambda x: 'CD' in str(x))))]

In [23]:
county_with_cd_nec_list = list(cd_abs_prov_prec['COUNTYFP'])

In [24]:
df['Pct'].str.slice(start = -5)[df['COUNTYFP'].isin(county_with_cd_nec_list)].value_counts()

CD 4)    177
CD 3)    164
D 11)    158
CD 7)    151
CD 8)     82
D 10)     69
CD 1)     61
CD 5)     48
CD 6)     38
CD 9)     33
CD 2)     30
Name: Pct, dtype: int64

In [25]:
#cd_abs_prov_prec['cd_countyfp'] = 
cd_abs_prov_prec['COUNTYFP'] + '-' + cd_abs_prov_prec['Pct'].str.slice(start = -5, stop = -1)

244     019-CD 5
245     019-CD 6
246     019-CD 5
247     019-CD 6
467     550-CD 3
          ...   
2425    177-CD 7
2494    800-CD 3
2495    800-CD 4
2496    800-CD 3
2497    800-CD 4
Length: 62, dtype: object

In [26]:
def add_cd_to_county(county_list, precinct, countyfp):
    if (countyfp in county_list):
        countyfp_cd = countyfp + '-' + precinct[-5:-1]
        return countyfp_cd
    else:
        countyfp_cd = countyfp
        return countyfp_cd

In [27]:
df['countyfp_cd'] = df.apply(lambda row: add_cd_to_county(county_with_cd_nec_list, row['Pct'], row['COUNTYFP']), axis = 1)

In [28]:
df[df['COUNTYFP'].isin(county_with_cd_nec_list)]

,Pct,LOCALITY,G16PREDCLI,G16PRERTRU,G16PRELJOH,G16PREIMCM,G16PREGSTE,G16PREOWRI,COUNTYFP,countyfp_cd
213,101 - Goodview Elem School (CD 5),Bedford County,310,1366,42,17,3,11,019,019-CD 5
214,102 - Hardy Vol Fire Co (CD 5),Bedford County,163,553,14,9,1,6,019,019-CD 5
215,103 - Chamblissburg Baptist Church (CD 5),Bedford County,263,942,45,8,5,5,019,019-CD 5
216,104 - Stewartsville Rescue Squad (CD 5),Bedford County,222,864,26,7,5,3,019,019-CD 5
217,201 - Bedford Christian Church (CD 5),Bedford County,155,454,19,9,4,5,019,019-CD 5
...,...,...,...,...,...,...,...,...,...,...
2493,706 - Huntersville (CD 3),Suffolk City,1488,271,26,13,20,15,800,800-CD 3
2494,AB - Central Absentee Precinct (CD 3),Suffolk City,2252,1151,69,22,15,25,800,800-CD 3
2495,Ab - Central Absentee Precinct (CD 4),Suffolk City,1365,568,28,6,3,2,800,800-CD 4
2496,Provisional (CD 3),Suffolk City,24,16,0,3,1,0,800,800-CD 3


In [29]:
df.countyfp_cd.value_counts()

059-D 11    117
810         100
107          95
059-CD 8     82
087-CD 7     69
           ... 
685           5
830           4
530           3
678           3
720           2
Name: countyfp_cd, Length: 148, dtype: int64

In [30]:
len(county_with_cd_nec_list)

62

In [31]:
df[df['COUNTYFP'] == '800']

,Pct,LOCALITY,G16PREDCLI,G16PRERTRU,G16PRELJOH,G16PREIMCM,G16PREGSTE,G16PREOWRI,COUNTYFP,countyfp_cd
2462,101 - Belleharbour (CD 3),Suffolk City,768,344,45,14,11,6,800,800-CD 3
2463,102 - Driver (CD 3),Suffolk City,886,1735,85,49,14,17,800,800-CD 3
2464,104 - Bennetts Creek (CD 3),Suffolk City,817,841,66,32,22,14,800,800-CD 3
2465,201 - Ebenezer (CD 3),Suffolk City,400,917,34,21,7,4,800,800-CD 3
2466,202 - Chuckatuck (CD 3),Suffolk City,507,821,40,10,8,6,800,800-CD 3
2467,203 - Kings Fork (CD 3),Suffolk City,1129,695,55,16,11,7,800,800-CD 3
2468,204 - Hillpoint (CD 3),Suffolk City,893,562,40,21,6,4,800,800-CD 3
2469,301 - White Marsh (CD 4),Suffolk City,333,174,5,5,1,0,800,800-CD 4
2470,302 - John F Kennedy (CD 3),Suffolk City,613,198,17,2,3,3,800,800-CD 3
2471,302 - John F Kennedy (CD 4),Suffolk City,886,22,6,3,4,2,800,800-CD 4


In [32]:
df[(df['Pct'].map(lambda x: 'Absentee' in str(x))) | (df['Pct'].map(lambda x: 'Provisional' in str(x)))]

,Pct,LOCALITY,G16PREDCLI,G16PRERTRU,G16PRELJOH,G16PREIMCM,G16PREGSTE,G16PREOWRI,COUNTYFP,countyfp_cd
17,Ab - Central Absentee Precinct,Accomack County,784,644,32,16,8,3,001,001
18,Provisional,Accomack County,3,1,2,0,0,0,001,001
47,Ab - Central Absentee Precinct,Albemarle County,4707,1828,238,120,39,61,003,003
48,Provisional,Albemarle County,48,29,5,2,2,1,003,003
76,Ab - Central Absentee Precinct,Alexandria City,15677,3069,484,251,138,321,510,510
...,...,...,...,...,...,...,...,...,...,...
2720,Provisional,Wise County,0,3,0,0,0,0,195,195
2732,Ab - Central Absentee Precinct,Wythe County,276,574,23,8,3,12,197,197
2733,Provisional,Wythe County,6,2,0,0,0,0,197,197
2747,Ab - Central Absentee Precinct,York County,1657,1712,150,71,16,38,199,199


### Real reallocation

In [34]:
absentee_and_prov = df[(df['Pct'].map(lambda x: 'Absentee' in str(x))) | (df['Pct'].map(lambda x: 'Provisional' in str(x)))]
groupby_absentee_and_prov_tot = absentee_and_prov.groupby(['countyfp_cd']).sum()
groupby_absentee_and_prov_tot

,G16PREDCLI,G16PRERTRU,G16PRELJOH,G16PREIMCM,G16PREGSTE,G16PREOWRI
countyfp_cd,,,,,,
001,787,645,34,16,8,3
003,4755,1857,243,122,41,62
005,204,332,10,2,4,0
007,293,515,14,7,0,3
009,519,654,20,12,7,10
...,...,...,...,...,...,...
800-CD 4,1366,570,28,6,3,2
810,14298,14090,966,268,172,251
820,333,344,26,3,1,7


In [35]:
groupby_county_df_tot = df.groupby(['countyfp_cd']).sum()
df_no_absent_or_provisional = df[(df['Pct'].map(lambda x: 'Absentee' not in str(x))) & (df['Pct'].map(lambda x: 'Provisional' not in str(x)))
                                & (df['LOCALITY'] != 'TOTALS')]
groupby_county_tot_no_absentee = df_no_absent_or_provisional.groupby('countyfp_cd').sum()

In [36]:
df_with_absentee_reallocated = df_no_absent_or_provisional.copy()
groupby_absentee_and_prov_tot.reset_index(inplace=True,drop=False)
groupby_county_tot_no_absentee.reset_index(inplace=True,drop=False)

to_dole_out_totals = groupby_absentee_and_prov_tot.copy()
precinct_specific_totals = groupby_county_tot_no_absentee.copy()

## PH CODE for vote allocation

#countyfp_cd
#df_no_absent_or_provisional

#Create some new columns for each of these races to deal with the allocation
for race in column_list:
    add_var = race+"_add"
    rem_var = race+"_rem"
    floor_var = race+"_floor"
    df_with_absentee_reallocated.loc[:,add_var]=0.0
    df_with_absentee_reallocated.loc[:,rem_var]=0.0
    df_with_absentee_reallocated.loc[:,floor_var]=0.0

#Iterate over the rows
#Note this function iterates over the dataframe two times so the rounded vote totals match the totals to allocate
for index, row in df_no_absent_or_provisional.iterrows():
    for race in column_list:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        #Grab the district
        county_id = row["countyfp_cd"]
        #Get the denominator for the allocation (the precinct vote totals)
        denom = precinct_specific_totals.loc[precinct_specific_totals["countyfp_cd"]==county_id][race]
        #Get one of the numerators, how many districtwide votes to allocate
        numer = to_dole_out_totals.loc[to_dole_out_totals["countyfp_cd"]==county_id][race]
        #Get the vote totals for this race in this precinct
        val = df_with_absentee_reallocated.at[index,race]
        #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
        if ((float(denom)==0)):
            vote_share = 0
        else:
            vote_share = (float(val)/float(denom))*float(numer)
        df_with_absentee_reallocated.at[index,add_var] = vote_share
        #Take the decimal remainder of the allocation
        df_with_absentee_reallocated.at[index,rem_var] = vote_share%1
        #Take the floor of the allocation
        df_with_absentee_reallocated.at[index,floor_var] = np.floor(vote_share)

#After the first pass through, get the sums of the races by district to assist in the rounding            
first_allocation = pd.DataFrame(df_with_absentee_reallocated.groupby(["countyfp_cd"]).sum())

#Now we want to iterate district by district to work on rounding
county_list = list(to_dole_out_totals["countyfp_cd"].unique()) 

#Iterate over the district
for county in county_list:
    for race in column_list:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
        to_go = int(np.round((int(to_dole_out_totals.loc[to_dole_out_totals["countyfp_cd"]==county][race])-first_allocation.loc[first_allocation.index==county,floor_var])))
        #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
        for index in df_with_absentee_reallocated.loc[df_with_absentee_reallocated["countyfp_cd"]==county][rem_var].nlargest(to_go).index:
            df_with_absentee_reallocated.at[index,add_var] = np.ceil(df_with_absentee_reallocated.at[index,add_var])

#Iterate over every race again
for race in column_list:
    add_var = race+"_add"
    #Round every allocation down to not add fractional votes
    df_with_absentee_reallocated.loc[:,add_var]=np.floor(df_with_absentee_reallocated.loc[:,add_var])
    df_with_absentee_reallocated.loc[:,race]+=df_with_absentee_reallocated.loc[:,add_var]

In [37]:
for val in column_list:
    vote_dif = df_with_absentee_reallocated[val].sum()-final[val].sum()
    if (vote_dif == 0):
        print(val+": EQUAL")
    else:
        print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES")
        
print("Counties with differences printed below:")


G16PREDCLI: EQUAL
G16PRERTRU: EQUAL
G16PRELJOH: EQUAL
G16PREIMCM: EQUAL
G16PREGSTE: EQUAL
G16PREOWRI: EQUAL
Counties with differences printed below:

All other races in all counties are equal


In [ ]:
diff_counties=[]
for i in column_list:
    diff = df_with_absentee_reallocated.groupby(["LOCALITY"]).sum()[i]-final.groupby(["LOCALITY"]).sum()[i]
    for val in diff[diff != 0].index.values.tolist():
        if val not in diff_counties:
            diff_counties.append(val)
    if len(diff[diff != 0]!=0):
        print(diff[diff != 0].to_string(header=False))
print("")
print("All other races in all counties are equal")

## Unique Identifier

In [ ]:
(df_with_absentee_reallocated['COUNTYFP'] + df_with_absentee_reallocated['Pct'].str.slice(stop = 3)).nunique()

In [ ]:
df_with_absentee_reallocated.shape[0]

In [ ]:
df_with_absentee_reallocated['vtdst'] = df_with_absentee_reallocated.Pct.str.slice(stop = 3)

In [ ]:
def vtdst_changer(vtdst):
    if (vtdst[1:3] == ' -'):
        two_lead_zero = '00' + vtdst[:1]
        return two_lead_zero
    elif (vtdst[1:3] == '- '):
        two_lead_zero = '00' + vtdst[:1]
        return two_lead_zero
    elif (vtdst[-1:] == ' '):
        one_lead_zero = '0' + vtdst[:2]
        return one_lead_zero
    elif (vtdst[-1:] == '-'):
        one_lead_zero = '0' + vtdst[:2]
        return one_lead_zero
    else:
        return vtdst

In [ ]:
df_with_absentee_reallocated['vtdst'] = df_with_absentee_reallocated['vtdst'].apply(vtdst_changer)
df_with_absentee_reallocated['unique_id'] = df_with_absentee_reallocated['COUNTYFP'] + df_with_absentee_reallocated['vtdst']

In [ ]:
final['unique_id'] = final['COUNTYFP'] + final['VTDST'].str.slice(start = 3)

In [ ]:
set(final['unique_id']) - set(df_with_absentee_reallocated['unique_id'])

In [ ]:
set(df_with_absentee_reallocated['unique_id']) - set(final['unique_id'])

In [ ]:
double_in_df = df_with_absentee_reallocated['unique_id'].value_counts()
df_double_list = double_in_df[double_in_df > 1].index
double_in_final = final['unique_id'].value_counts()
final_double_list = double_in_final[double_in_final > 1].index
print(set(final_double_list) - set(df_double_list))
print(set(df_double_list) - set(final_double_list))

In [ ]:
len(final_double_list)

In [ ]:
len(df_double_list)

In [ ]:
def vtdst_add_cd(unique_id, precinct, double_list):
    if (unique_id in double_list):
        unique_id = unique_id + precinct[-6:]
        return unique_id
    else:
        return unique_id

In [ ]:
df_with_absentee_reallocated['unique_id_1'] = df_with_absentee_reallocated.apply(lambda row: vtdst_add_cd(row['unique_id'], row['Pct'], list(df_double_list)), axis = 1)
df_with_absentee_reallocated['unique_id_1'].nunique()

In [ ]:
final['unique_id'].nunique()

In [ ]:
final['unique_id_1'] = final.apply(lambda row: vtdst_add_cd(row['unique_id'], row['PRECINCT'], final_double_list), axis = 1)
final['unique_id_1'].nunique()

In [ ]:
print(set(final['unique_id_1']) - set(df_with_absentee_reallocated['unique_id_1']))
print(set(df_with_absentee_reallocated['unique_id_1']) - set(final['unique_id_1']))

# Shapefile

In [ ]:
county_fips = []
for directory in os.listdir('./vest_va_2016/census_shps_by_county_all_unzip/'):
    if not directory[0] == '.':
        county_fips.append(directory[-5:])
        
proj = final.crs   

county_vtds = []
for i in county_fips: #i dont have fips_codes file
    ref = './vest_va_2016/census_shps_by_county_all_unzip/partnership_shapefiles_19v2_'
    vtd_ref = ref + i + '/PVS_19_v2_vtd_' + i + '.shp' 
    vtd_shp = gpd.read_file(vtd_ref)
    print(i, ' added to list')
    county_vtds.append(vtd_shp)

global shp
shp = gpd.GeoDataFrame(pd.concat(county_vtds, axis = 0) , crs = proj) 

In [ ]:
shp.plot()
final.plot()

In [ ]:
shp.shape[0]

In [ ]:
shp['unique_id'] = shp['COUNTYFP'] + shp['VTDST'].str.slice(start = 3)

In [ ]:
final.shape[0]

In [ ]:
print(len((set(shp.COUNTYFP + shp.VTDST.str.slice(start = 3)) - set(final.COUNTYFP + final.VTDST.str.slice(start = 3)))))
print(len((set(final.COUNTYFP + final.VTDST.str.slice(start = 3)) - set(shp.COUNTYFP + shp.VTDST.str.slice(start = 3)))))

In [ ]:
shp['unique_id'].nunique()

In [ ]:
#Dictionary based on comparison in Excel sheet + plotting unique ids to observe match or not
overlay_w_shp_to_df_mod_dict = {
    '059925': '059925',
    '077011': '077401',
    '077012': '077401',
    '095041': '095104',
    '095042': '095104',
    '107422': '107419',
    '153061': '153106',
    '191021': '191702',
    '191022': '191702',
    '520041':  '520004',
    '520042':  '520004',
    '550019': '550019',
    '550045': '550045',
    '685031': '685003',
    '685032': '685003',
    '800403': '800403'
}

In [ ]:
set(overlay_w_shp_to_df_mod_dict.values()) - (set(final['unique_id']) - set(shp['unique_id']) )

In [ ]:
set(overlay_w_shp_to_df_mod_dict.keys()) - (set(shp['unique_id']) - set(final['unique_id']))

In [ ]:
set(df_with_absentee_reallocated['unique_id_1']) - set(final['unique_id_1'])

In [ ]:
set(final['unique_id_1']) - set(df_with_absentee_reallocated['unique_id_1'])

### Merge in df to match final

In [ ]:
merge_in_df_match_final = {'059925CD 10)': '059925',
 '059925CD 11)': '059925',
 '550019(CD 3)': '550019',
 '550019(CD 4)': '550019',
 '550045(CD 3)': '550045',
 '550045(CD 4)': '550045',
 '800403(CD 3)': '800403',
 '800403(CD 4)': '800403'}

In [ ]:
df_with_absentee_reallocated.loc[df_with_absentee_reallocated['unique_id_1'].isin(merge_in_df_match_final.keys()), 'unique_id_1'] = df_with_absentee_reallocated.loc[df_with_absentee_reallocated['unique_id_1'].isin(merge_in_df_match_final.keys()), 'unique_id_1'].map(merge_in_df_match_final)

In [ ]:
print(set(df_with_absentee_reallocated['unique_id_1']) - set(final['unique_id_1']))
print(set(final['unique_id_1']) - set(df_with_absentee_reallocated['unique_id_1']))

In [ ]:
df_with_absentee_reallocated['unique_id_1'].nunique()

### Check value counts in unique_id_1 column to sum election results into merge

In [ ]:
double_in_df_1 = df_with_absentee_reallocated['unique_id_1'].value_counts()
df_double_list_1 = double_in_df_1[double_in_df_1 > 1].index
df_with_absentee_reallocated[df_with_absentee_reallocated['unique_id_1'].isin(df_double_list_1)]

In [ ]:
#remove rows with zero votes
df_remove_list = ['19 - Jolliff (CD 3)', '45 - Sunray Ii (CD 3)', '925 - Old Mill (CD 11)', '403 - Southside (CD 3)']
df_with_absentee_reallocated[df_with_absentee_reallocated['Pct'].isin(df_remove_list)]

In [ ]:
df_with_absentee_reallocated = df_with_absentee_reallocated[~df_with_absentee_reallocated['Pct'].isin(df_remove_list)]

In [ ]:
df_with_absentee_reallocated.shape

### Now that df and final match up, want to match up df/vest and raw shp

In [ ]:
len(set(df_with_absentee_reallocated['unique_id_1']) - set(shp['unique_id']))

In [ ]:
len(set(shp['unique_id']) - set(df_with_absentee_reallocated['unique_id_1']))

In [ ]:
tidy_cd_id_format_dict = {
 '059513CD 11)': '059513(CD 11)',
 '153110CD 10)': '153110(CD 10)',
 '153112CD 10)': '153112(CD 10)',
 '153210CD 11)': '153210(CD 11)',
 '153312CD 11)': '153312(CD 11)',
 '153609CD 11)': '153609(CD 11)',
}

In [ ]:
df_with_absentee_reallocated.loc[df_with_absentee_reallocated['unique_id_1'].isin(tidy_cd_id_format_dict.keys()), 'unique_id_1'] = df_with_absentee_reallocated.loc[df_with_absentee_reallocated['unique_id_1'].isin(tidy_cd_id_format_dict.keys()), 'unique_id_1'].map(tidy_cd_id_format_dict)
final.loc[final['unique_id_1'].isin(tidy_cd_id_format_dict.keys()), 'unique_id_1'] = final.loc[final['unique_id_1'].isin(tidy_cd_id_format_dict.keys()), 'unique_id_1'].map(tidy_cd_id_format_dict)

In [ ]:
set(final['unique_id_1']) - set(df_with_absentee_reallocated['unique_id_1'])

In [ ]:
final[final['unique_id_1'] == '059700']

### Now that id format tidied, load in CD in shp to create matching unique id

#### **Load in cd shp to enable splits later on**

In [ ]:
county_cd = []

for i in county_fips: #i dont have fips_codes file
    ref = './vest_va_2016/census_shps_by_county_all_unzip/partnership_shapefiles_19v2_'
    cd_ref = ref + i + '/PVS_19_v2_cd_' + i + '.shp' 
    cd_shp = gpd.read_file(cd_ref)
    print(i, ' added to list')
    county_cd.append(cd_shp)
global cd
cd = gpd.GeoDataFrame(pd.concat(county_cd, axis = 0) , crs = proj) 

In [ ]:
cd.plot()
overlay = gpd.overlay(cd, shp, how = 'union', make_valid = True, keep_geom_type = True)
overlay.plot()

In [ ]:
overlay_w_shp = gpd.GeoDataFrame(pd.merge(overlay, shp, on = 'unique_id', how = 'outer'), crs = proj)
overlay_w_shp.shape

In [ ]:
overlay_w_shp['unique_id_1'] = overlay_w_shp['unique_id'] + '(CD ' +overlay_w_shp['CDFP'].str.lstrip('0') + ')'

In [ ]:
len(df_double_list)

In [ ]:
overlay_w_shp.loc[~overlay_w_shp['unique_id'].isin(df_double_list), 'unique_id_1'] = overlay_w_shp.loc[~overlay_w_shp['unique_id'].isin(df_double_list), 'unique_id']

### See how well id matches up to df

In [ ]:
len(set(overlay_w_shp['unique_id_1']) - set(df_with_absentee_reallocated['unique_id_1']))

In [ ]:
len(set(df_with_absentee_reallocated['unique_id_1']) - set(overlay_w_shp['unique_id_1']))

In [ ]:
list(overlay_w_shp['NAME'][overlay_w_shp.unique_id_1.isin(set(overlay_w_shp['unique_id_1']) - set(df_with_absentee_reallocated['unique_id_1']))])

In [ ]:
list(final['PRECINCT'][final.unique_id_1.isin((set(final['unique_id_1']) - set(overlay_w_shp['unique_id_1'])))])

**Sets match up well with observed from US house file. With that, can use same dictionary with slight modification to achieve results possible.**

In [ ]:
overlay_w_shp_to_df_dict = {
    '059925(CD 10)': '059925',
    '077011': '077401',
    '077012': '077401',
    '095041': '095104',
    '095042(CD 2)': '095104',
    '107422': '107419',
    '153061': '153106',
    '191021': '191702',
    '191022': '191702',
    '520041':  '520004',
    '520042':  '520004',
    '550019(CD 4)': '550019',
    '550045(CD 4)': '550045',
    '685031': '685003',
    '685032': '685003',
    '800403(CD 4)': '800403'
}

In [ ]:
len(overlay_w_shp_to_df_dict)

In [ ]:
overlay_w_shp.loc[overlay_w_shp['unique_id_1'].isin(overlay_w_shp_to_df_dict.keys()), 'unique_id_1'] = overlay_w_shp.loc[overlay_w_shp['unique_id_1'].isin(overlay_w_shp_to_df_dict.keys()), 'unique_id_1'].map(overlay_w_shp_to_df_dict)

In [ ]:
final[final['unique_id_1'].isin(set(final['unique_id_1']) - set(overlay_w_shp['unique_id_1']))].plot()
overlay_w_shp['geometry_x'][overlay_w_shp['unique_id_1'].isin(set(overlay_w_shp['unique_id_1']) - set(final['unique_id_1']))].plot()

- load into QGIS to isolate that point - here probs dont worry about
    - add as part of internal knowledge base
moving forward
- code to find intersection values 
    - we were not able to match to its own precicnt, but land here is split in VEST's 
    - eventually make fxn to show all alignments of what VEST did 

In [ ]:
overlay_w_shp['unique_id_1'].value_counts()

In [ ]:
overlay_w_shp.columns

In [ ]:
overlay_w_shp['geometry'] = overlay_w_shp['geometry_x']
#overlay_w_shp = overlay_w_shp.drop(['geometry_x', 'geometry_y'], axis = 1)

In [ ]:
overlay_w_shp[['geometry','geometry_x', 'geometry_y']][overlay_w_shp['geometry_x'] == None]

In [ ]:
overlay_w_shp.loc[overlay_w_shp['geometry_x'] == None, 'geometry'] = overlay_w_shp.loc[overlay_w_shp['geometry_x'] == None, 'geometry_y']

In [ ]:
overlay_w_shp[['geometry','geometry_x', 'geometry_y']][overlay_w_shp['geometry_x'] == None]

In [ ]:
overlay_w_shp.columns

In [ ]:
overlay_w_shp = overlay_w_shp.dissolve(by = 'unique_id_1', as_index = False)

In [ ]:
overlay_w_shp.unique_id_1.nunique()

In [ ]:
len(set(df_with_absentee_reallocated.unique_id_1) -set(overlay_w_shp.unique_id_1))

In [ ]:
len(set(overlay_w_shp.unique_id_1) - set(df_with_absentee_reallocated.unique_id_1))

In [ ]:
len(final_double_list)

In [ ]:
final.shape

**Questions for Peter: 5/11/21**
- do you think I can expect any better of a match rate? - no
- the one spot bottom left, any way to identify which precinct that is and match it up?
- inner or outer join for recreation of election results and shapefile?
    - outer to see what is joined and not joined
    - print merge.value_counts(
    - allows to show how many only have 1 vs other
- Anything else noteworthy before all laid out in report?
- Should I use the new template? 


- I think I still need to dissolve ? Why didnt it work?

**NEXT 5/11/21**
- clean up code for US president (here) and US house
- modify special elections scripts to validate based on joined file
- write report (+ readme?)
- upload to AWS
- send to Peter for review

# Join shapefile and election results

In [ ]:
shp_df_merge = pd.merge(overlay_w_shp, df_with_absentee_reallocated, on = 'unique_id_1', how = 'inner', suffixes = ['_x', '_y'])
shp_df_gdf = gpd.GeoDataFrame(shp_df_merge, geometry = 'geometry')

In [ ]:
shp_df_gdf.plot()
final.plot()

In [ ]:
shp_df_gdf.columns

In [ ]:
shp_df_gdf = shp_df_gdf.drop(['geometry_x', 'geometry_y'], axis = 1)

# Validation

## Shapefile

In [ ]:
shp_final_merge = pd.merge(shp_df_gdf, final, on = 'unique_id_1', how = 'outer', suffixes = ['_x', '_y'],indicator=True)
overlay_w_shp['geometry'].geom_almost_equals(final['geometry'], decimal = 0)

In [ ]:
both = shp_final_merge[shp_final_merge["_merge"]=="both"]
source_geoms = gpd.GeoDataFrame(both,geometry="geometry_x",crs=final.crs)
vest_geoms = gpd.GeoDataFrame(both,geometry="geometry_y",crs=final.crs)
source_geoms = source_geoms.to_crs(3857)
vest_geoms = vest_geoms.to_crs(3857)
source_geoms["geometry_x"]=source_geoms.buffer(0)
vest_geoms["geometry_y"]=vest_geoms.buffer(0)
vals = source_geoms.geom_almost_equals(vest_geoms,decimal=0)
print(vals.value_counts())

In [ ]:
count = 0
area_list = []
big_diff = pd.DataFrame(columns=["area"])
for i in range(0,len(source_geoms)):
    diff = source_geoms.iloc[[i]].symmetric_difference(vest_geoms.iloc[[i]])
    intersection = source_geoms.iloc[[i]].intersection(vest_geoms.iloc[[i]])
    area = float(diff.area/10e6)
    area_list.append(area)
    #print("Area is " + str(area))

    if (area > 1):
        count += 1
        name = source_geoms.at[i,"Pct"]
        big_diff.loc[name]=area
        print(str(count)+") For " + name + " difference in area is " + str(area))
        if (intersection.iloc[0].is_empty):
            base = diff.plot(color="red")
            source_geoms.iloc[[i]].plot(color="orange",ax=base)
            vest_geoms.iloc[[i]].plot(color="blue",ax=base)
            base.set_title(name)
        else:
            base = diff.plot(color="red")
            source_geoms.iloc[[i]].plot(color="orange",ax=base)
            vest_geoms.iloc[[i]].plot(color="blue",ax=base)
            intersection.plot(color="green",ax=base)
            base.set_title(name)

In [ ]:
df = pd.DataFrame(area_list)
print(df.shape)

print(str(len(df[df[0]==0]))+" precincts w/ a difference of 0 km^2")
print(str(len(df[(df[0]<.1) & (df[0]>0)]))+ " precincts w/ a difference between 0 and .1 km^2")
print(str(len(df[(df[0]<.5) & (df[0]>=.1)]))+ " precincts w/ a difference between .1 and .5 km^2")
print(str(len(df[(df[0]<1) & (df[0]>=.5)]))+ " precincts w/ a difference between .5 and 1 km^2")
print(str(len(df[(df[0]<2) & (df[0]>=1)]))+ " precincts w/ a difference between 1 and 2 km^2")
print(str(len(df[(df[0]<5) & (df[0]>=2)]))+ " precincts w/ a difference between 2 and 5 km^2")
print(str(len(df[(df[0]>=5)]))+ " precincts w/ a difference greater than 5 km^2")

In [ ]:
shp_final_merge.columns

In [ ]:
shp_final_merge.shape

In [ ]:
final[final['unique_id'].isin(final_double_list)]

## Election result comparison

In [ ]:
final.columns

In [ ]:
column_list = ['G16PREDCLI', 'G16PRERTRU', 'G16PRELJOH', 'G16PREIMCM', 'G16PREGSTE', 'G16PREOWRI']

In [ ]:
final_shp_merge = pd.merge(shp_df_gdf, final, on = 'unique_id_1', how = 'inner', suffixes = ('_x', '_y'))

In [ ]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff >0):
                if(diff>0):
                    print(i, "{:.>72}".format(df.iloc[j]["Pct"]), "(V)","{:.>5}".format(int(df.iloc[j][right_Data]))," (S){:.>5}".format(int(df.iloc[j][left_Data])),"(D):{:>5}".format(int(df.iloc[j][right_Data])-int(df.iloc[j][left_Data])))           
                #print(df.iloc[j]['countypct'])
                
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]["Pct"])
                    print(df.iloc[j][left_Data])
                    print(df.iloc[j][right_Data])
                if (diff>max_diff):
                    max_diff = diff
                    #print("New max diff is: ", str(max_diff))
                    #print(df.iloc[j]['cty_pct'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]["Pct"])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    diff_list.sort()
    print(diff_list)

In [ ]:
validater_row(final_shp_merge, column_list)

In [ ]:
final_df_merge = pd.merge(df_with_absentee_reallocated, final, on = 'unique_id_1', how = 'inner', suffixes = ('_x', '_y'))

In [ ]:
validater_row(final_df_merge, column_list)